In [ ]:
# aid.ipynb
# Author: Kyle Larson
# Purpose top secret
import numpy as np 
import soundfile as sf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# Stack overflow was consulted in order to parse '.flac' 
path = '../input/rfcx-species-audio-detection/train/00204008d.flac'
data, samplerate = sf.read(path) #this method is thanks to Harry Moreno 
#from his Stack Overflow response on June 13 '18 at the following URL:
#https://stackoverflow.com/questions/50804170/load-flac-file-in-python-same-as-scipy-or-librosa
species_data = pd.DataFrame(data)
species_data.head()